In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from IPython.display import clear_output
import time
import copy
np.seterr(divide='ignore', invalid='ignore')

#import import_ipynb
import nbimporter
from Scoring_Function import score

low_memory = False

# Read csv into pandas dataframe
df = pd.read_csv('train_fixed.csv', low_memory=False)

from IPython.display import Audio
sound_file = 'Super Mario Bros. - Coin Sound Effect.mp3'

Importing Jupyter notebook from Scoring_Function.ipynb


In [16]:
class Player:
    
    # Create player object with attributes that can be mutated for player movement
    def __init__(self, player_row, increment, tkl_score, rush_slow, def_target_rush, 
                 def_target_thresh, block_slow, block_time, rush_vision, time_pre_dyn, max_speed,
                 lbs_force):
        # Parameters
        self.increment = increment
        self.tkl_score = tkl_score
        self.rush_slow = rush_slow
        self.def_target_rush = def_target_rush
        self.def_target_thresh = def_target_thresh
        self.block_slow = block_slow
        self.block_time = block_time
        self.rush_vision = rush_vision
        self.time_pre_dyn = time_pre_dyn
        self.MAXIMUM_SPEED = max_speed
        self.LBF = lbs_force
        
        self.time = 0
        self.play_ID = player_row['PlayId']
        self.play_direct = player_row['PlayDirection']
        self.x = player_row['X']
        self.y = player_row['Y']
        self.target_x = None
        self.target_y = None
        self.speed = player_row['S']
        self.accel = player_row['A'] # yd/s^2
        self.orient = player_row['Orientation']
        self.direct = player_row['Dir']
        if self.direct > 270:
            self.direct_rad = math.radians(360 - (self.direct - 90))
        else:
            self.direct_rad = math.radians(-(self.direct - 90))
        self.ID = player_row['NflId']
        self.rush_ID = player_row['NflIdRusher']
        self.weight = player_row['PlayerWeight'] # pounds
        self.position = player_row['Position']
        self.v = np.array([self.speed * math.cos(self.direct_rad), self.speed * math.sin(self.direct_rad)])
        self.a = np.array([self.accel * math.cos(self.direct_rad), self.accel * math.sin(self.direct_rad)])
        self.new_accel = self.LBF / (32.174 * self.weight * 3) # yd/s^2 with proper conversions
        self.tackle_attempted = False
        self.tackled = False
        self.blocked = False
        self.block_timer = 0.0
        self.reached_path = False
        self.slowed = False
        if player_row['Team'] == 'home':
            self.team = player_row['HomeTeamAbbr']
        else:
            self.team = player_row['VisitorTeamAbbr']
        if self.team == player_row['PossessionTeam']:
            self.side = 'O'
        else:
            self.side = 'D'
        if self.isRusher(): # If the player is the rusher
            self.color = 'Orange' 
            if self.play_direct == 'left': # set target location to run towards if left
                self.target_x = 10
                self.target_y = self.y
            else:                          # set target location to run towards if right
                self.target_x = 110
                self.target_y = self.y
        elif self.side == 'O': # If the player is on offense
            self.color = 'Blue'
            if self.play_direct == 'left': # set target location to run towards if left
                self.target_x = 10
                self.target_y = self.y
            else:                          # set target location to run towards if right
                self.target_x = 110
                self.target_y = self.y
        else: # If the player is on defense
            self.color = 'Red'
            self.target_x = None
            self.target_y = None
            
        if self.position in ['FS', 'S', 'SAF', 'SS']:
            self.is_safety = True
        else:
            self.is_safety = False
            
        if self.position in ['WR']:
            self.block_time = 0.5
            
    # Update player's x, y, speed, acceleration, direction, and orientation using target_x and target_y
    def update(self):
        
        if self.time < self.time_pre_dyn: # This value is how long(seconds) someone runs in a direction before pursuing target

            if self.is_safety: # If safetys, don't move
                self.v *= self.block_slow
            else:
                self.x += self.v[0] * self.increment
                self.y += self.v[1] * self.increment
                self.v += self.a * self.increment
                s = np.linalg.norm(self.v)
                self.v /= s
                self.v *= min(s, self.MAXIMUM_SPEED)
        else:
            self.a = np.array([self.target_x - self.x, self.target_y - self.y])
            a = np.linalg.norm(self.a)
            self.a /= a
            self.a *= self.new_accel
            
            self.v += self.a * self.increment
            s = np.linalg.norm(self.v)
            self.v /= s
            self.v *= min(s, self.MAXIMUM_SPEED)
            
            if not self.blocked:
                self.x += self.v[0] * self.increment
                self.y += self.v[1] * self.increment
                        
        self.time += self.increment
        
    # Update player's target point to run towards
    def set_target(self, rusher = None, defense = None, rusher_progression = 0): # changed
        if rusher == None:
            if self.isRusher(): # Rusher's adjustment
                self.target_y = self.y
                
                in_path = True
                count = 0
                while (in_path): # loop update path
                    y_offset = []
                    temp_y = self.target_y
                    for player in defense: # find distance to each defensive player
                        if self.play_direct == 'left':
                            curr_dist = self.x - player.x
                        else:
                            curr_dist = player.x - self.x
                        if curr_dist < self.rush_vision and curr_dist > 0: # only consider evading defenders < 5 yards in front
                            slope = (self.target_y - self.y) / (self.target_x - self.x)
                            intercept = self.y - slope * self.x
                            A = slope
                            B = -1
                            C = intercept
                            dist_to_path = abs(A * player.x + B * player.y + C) / math.sqrt(A**2 + B**2)
                            if dist_to_path < 1.0: # distance from rusher's path
                                y_offset.append(player)
                    count += 1
                
                    if len(y_offset) > 0 and count < 10: # if there are defenders to evade
                        # update path upwards or downwards based on current velocity
                        if self.v[1] < 0: # if running down
                            self.target_y -= 7.0
                        else: # if running up
                            self.target_y += 7.0
                    else:
                        in_path = False
                
            else: # Offense adjustment
                dist = 1000
                closest = None
                for player in defense:
                    if self.position == 'QB': # QB doesn't block
                        continue
                    curr_dist = np.linalg.norm(np.array([player.x - self.x, player.y - self.y]))
                    if curr_dist < dist: # Find closest defender
                        dist = curr_dist
                        self.target_x = player.x
                        self.target_y = player.y
                        closest = player
                        
                        v1 = closest.v
                        v2 = [self.x - closest.x, self.y - closest.y]
                        cosang = np.dot(v1, v2)
                        sinang = np.linalg.norm(np.cross(v1, v2))
                        ang =  np.arctan2(sinang, cosang)
                        
                if dist < 1 and (ang < 1.4 or rusher_progression < 0):#self.block_timer < self.block_time and self.time < 1.5: # changed
                    closest.v *= self.block_slow # Block defensive player and slow them down
                    self.v *= 0
                    
                    closest.blocked = True
                    self.block_timer += self.increment
                    self.blocked = True
                else:
                    if closest != None:
                        closest.blocked = False
                        self.blocked = False

        else: # Defense adjustment
            curr_dist = np.linalg.norm(np.array([rusher.x - self.x, rusher.y - self.y]))
            
            v1 = rusher.v
            v2 = [self.x - rusher.x, self.y - rusher.y]
            cosang = np.dot(v1, v2)
            sinang = np.linalg.norm(np.cross(v1, v2))
            ang =  np.arctan2(sinang, cosang)
            
            if curr_dist < self.def_target_thresh and ang < 1.0: # if < X yards from rusher, run at rusher # changed
                self.target_x = rusher.x
                self.target_y = rusher.y
                self.v *= 0.5
                if curr_dist < 0.8 or ang < 0.23:
                    if not self.slowed:
                        self.slowed = True
                        self.v *= 0.2
            else:
                rush_direct_norm = rusher.v / np.linalg.norm(rusher.v)
                self.target_x = rusher.x + rush_direct_norm[0] * self.def_target_rush
                self.target_y = rusher.y + rush_direct_norm[1] * self.def_target_rush
                
                if self.position in ['CB', 'FS', 'S', 'SAF', 'SS']:
                        
                    v1_norm = np.linalg.norm(v1)     
                    proj_of_v2_on_v1 = (np.dot(v2, v1)/v1_norm**2)*v1
                    new_vec = proj_of_v2_on_v1 / math.cos(ang)
                    temp_target_x = rusher.x + new_vec[0]
                    temp_target_y = rusher.y + new_vec[1]
                    new_vec *= np.linalg.norm(np.array([temp_target_x - self.x, temp_target_y - self.y])) / curr_dist
    
                    if ang < 0.23:
                        self.v *= 0.3
                                    
                    if ang > 0.15:
                        self.target_x = rusher.x + new_vec[0]
                        self.target_y = rusher.y + new_vec[1]
                    else:
                        if not self.reached_path:
                            self.reached_path = True
                            self.v *= 0
                        self.target_x = rusher.x
                        self.target_y = rusher.y
        
    # Simulate player's attempt to tackle the rusher
    def tackle(self, rusher):
        
        #tkl_score = self.weight / (self.weight + rusher.weight) # Weight based probability to tackle
        tkl_score = self.tkl_score
        
        if self.is_safety: # Safety guaranteed to tackle
            tkl_score = 1
        outcome = np.random.choice(np.arange(0, 2), p=[1 - tkl_score, tkl_score])
        
        if outcome == 1:
            rusher.tackled = True
        else:
            slow_factor = np.random.choice(np.arange(0,2)) # Randomize slow
            
            rusher.v *= (self.rush_slow + slow_factor * 0.1) # slow down rusher
        
        self.tackle_attempted = True
        
    # Check if this is rusher
    def isRusher(self):
        return self.ID == self.rush_ID
    

In [17]:
class Play:
    def __init__(self, play_ID, increment=0.1, tkl_rad=0.8, tkl_score=0.5, rush_slow=0.55, 
                 def_target_rush=3, def_target_thresh=2.5, block_slow=0.0, block_time=1.0, rush_vision=5,
                 time_pre_dyn=0.7, max_speed=9.5, lbs_force=190000):
        # Parameters
        self.increment = increment
        self.tkl_rad = tkl_rad # Distance to rusher before attempting tackle
        self.tkl_score = tkl_score # Probability of defender tackling rusher
        self.rush_slow = rush_slow # Slow down rusher after missed tackle
        self.def_target_rush = def_target_rush # Distance ahead of rusher defender targets rusher(cut off rusher)
        self.def_target_thresh = def_target_thresh # Distance before defender targets rusher
        self.block_slow = block_slow # Amount blocking slows down defender
        self.block_time = block_time # Time spent blocking for a defender
        self.rush_vision = rush_vision # Rusher's vision down field
        self.time_pre_dyn = time_pre_dyn # Time elapsed before dynamic simluation
        self.max_speed = max_speed # Maximum speed for each player
        self.lbs_force = lbs_force # Pound force attribute for each player used in calculating acceleration
        
        self.field_set = False
        self.play_ID = play_ID
        self.players_df = df[df['PlayId'] == play_ID]
        self.players_list = []
        self.yards = 0
        self.yards_actual = self.players_df['Yards'].iloc[0]
        
        self.rush_index = 0
        self.count = 0
        for row in self.players_df.iterrows():
            self.count += 1
            if row[1]['NflId'] == row[1]['NflIdRusher']:
                self.rush_index = self.count
                self.rusher = Player(row[1], self.increment, self.tkl_score, self.rush_slow, self.def_target_rush, 
                                             self.def_target_thresh, self.block_slow, self.block_time,
                                             self.rush_vision, self.time_pre_dyn, self.max_speed, self.lbs_force)
                self.pos_team = row[1]['PossessionTeam']
                self.field_pos = row[1]['FieldPosition']
            else:
                self.players_list.append(Player(row[1], self.increment, self.tkl_score, self.rush_slow, 
                                                        self.def_target_rush, self.def_target_thresh, self.block_slow,
                                                        self.block_time, self.rush_vision, self.time_pre_dyn, 
                                                        self.max_speed, self.lbs_force))
        
        if self.rusher.play_direct == 'right':
            if not isinstance(self.field_pos, str):
                self.scrimmage = 60
            elif self.pos_team == self.field_pos:
                self.scrimmage = self.players_df['YardLine'].iloc[0] + 10
            else:
                self.scrimmage = 120 - (self.players_df['YardLine'].iloc[0] + 10)
        else:
            if not isinstance(self.field_pos, str):
                self.scrimmage = 60
            elif self.pos_team == self.field_pos:
                self.scrimmage = 120 - (self.players_df['YardLine'].iloc[0] + 10)
            else:
                self.scrimmage = self.players_df['YardLine'].iloc[0] + 10
              
        if self.rush_index < 12:
            offense = self.players_list[:10]
            defense = self.players_list[10:]
        else:
            offense = self.players_list[11:]
            defense = self.players_list[:11]
        
        for player in defense:
            if player.is_safety and abs(player.x - self.scrimmage) < 8:
                player.is_safety = False
                
        self.rusher_progression = None # changed
               
    def plot(self):
        
        self.setup_field()
        
        for player in self.players_list:
            # plot player location
            plt.scatter(player.x, player.y, s = 100, color = player.color)

            # plot directional vector at start of play for player
            # plt.quiver(player.x, player.y, player.v[0], player.v[1], width = 0.001)
            
        # plot rusher location
        plt.scatter(self.rusher.x, self.rusher.y, s = 100, color = self.rusher.color)

        # plot directional vector for rusher
        # plt.quiver(self.rusher.x, self.rusher.y, self.rusher.v[0], self.rusher.v[1], width = 0.001)
        
        # plot target location for rusher
        plt.scatter(self.rusher.target_x, self.rusher.target_y, s = 200, color = 'Green')
        
        plt.plot([self.rusher.x, self.rusher.target_x], [self.rusher.y, self.rusher.target_y], 'ro-', color = "Orange")
        
    def plot_rusher(self):
        
        self.setup_field()
        
        # plot rusher location
        plt.scatter(self.rusher.x, self.rusher.y, s = 100, color = self.rusher.color)

        # plot directional vector for rusher
        # plt.quiver(self.rusher.x, self.rusher.y, self.rusher.v[0], self.rusher.v[1], width = 0.001)
        
        # plot target location for rusher
        plt.scatter(self.rusher.target_x, self.rusher.target_y, s = 200, color = 'Green')
    
        
    def setup_field(self):
        plt.figure(figsize=(20,10))
        plt.ylim(top = 53.3, bottom = 0)
        plt.xlim(left = 0, right = 120)
        plt.axvline(x = self.scrimmage, label = "Line of Scrimmage", ls = "--", color = 'Blue')#plot line of scrimmage
        plt.axvline(x = 10, label = "Left Endzone", ls = "-", color = 'Red')#plot left endzone
        plt.axvline(x = 110, label = "Right Endzone", ls = "-", color = 'Red')#plot right endzone
        plt.text(120 - (self.players_df['YardLine'].iloc[0] + 10) + plt.xlim()[1] * 0.01 , plt.ylim()[1] * 0.01,\
                    str(self.players_df['YardLine'].iloc[0]),\
                    horizontalalignment='center', fontweight='bold', color='black')
        plt.legend()
        
    def update(self):
        if self.rush_index < 12:
            offense = self.players_list[:10]
            defense = self.players_list[10:]
        else:
            offense = self.players_list[11:]
            defense = self.players_list[:11]
            
        self.rusher.set_target(None, defense)
        self.rusher.update()
        
        if self.rusher.play_direct == 'right': # changed
            self.rusher_progression = self.rusher.x - self.scrimmage
        else:
            self.rusher_progression = self.scrimmage - self.rusher.x
        
        for player in offense:
            player.set_target(None, defense, self.rusher_progression) # changed
            player.update()
        
        for player in defense:
            if player.tackle_attempted:
                continue
            
            player.set_target(self.rusher, defense)
            player.update()
            
            dist = np.linalg.norm(np.array([player.x - self.rusher.x, player.y - self.rusher.y]))
            if dist < self.tkl_rad and not player.blocked:# tackle radius
                player.tackle(self.rusher)

In [18]:
def simulate_play(play_ID, params):
    par = copy.deepcopy(params)
    time_len_play = 20
        
    play = Play(play_ID, **par)
    
    for i in range(int(time_len_play / play.increment)):
        play.update()
        if play.rusher.tackled:
            break
        if play.rusher.play_direct == 'right' and (play.rusher.x > 110):
            break
        if play.rusher.play_direct == 'left' and (play.rusher.x < 10):
            break
    
    if play.rusher.play_direct == 'right':
        play.yards = play.rusher.x - play.scrimmage
    else:
        play.yards = play.scrimmage - play.rusher.x
    
    return round(play.yards)

def simulate_play_mult(play_ID, num_times, params):
    preds_for_play = []
    for i in range(num_times):
        preds_for_play.append(simulate_play(play_ID, params))
    dist = np.histogram(preds_for_play, bins = 199, range = (-99, 99))[0] / num_times #find probability for each bin
    cumul_sum = np.cumsum(dist)
    cumul_sum_play = np.insert(cumul_sum, 0, None)
    return cumul_sum_play


# Wrapper Function to enter parameters and dataset(train/test) to run on

def score_iteration(data, params):
    sample_sub = pd.read_csv('sample_submission.csv').rename(columns={"Unnamed: 0": "PlayId"}) #bring in sample sumbission
    sub = sample_sub.set_index('PlayId').drop([0])
    
    plays = data # data is the set of unique play ID's to run simulations on 
    
    num_sims = 10
    if 'num_sims' in params:
        num_sims = params['num_sims']
        del params['num_sims']

    for i in range(plays.size):
        sub.loc[plays[i]] = simulate_play_mult(plays[i], num_sims, params)[1:]

    sub = sub.reset_index()
#     pd.set_option('display.max_columns', None)
#     print(sub)
    return score(sub, df) # Return the score


In [25]:
all_plays = pd.unique(df['PlayId'])

#params = {'tkl_rad':1.5} Example
params = {'num_sims':100}
score_iteration(all_plays[2:3], params)

0.06030150753768844

In [314]:
Audio(sound_file, autoplay=True)

In [15]:
all_plays[0:10]

array([20170907000118, 20170907000139, 20170907000189, 20170907000345,
       20170907000395, 20170907000473, 20170907000516, 20170907000653,
       20170907000680, 20170907000801])

In [ ]:
0-10: 0.01316
10-20: 0.0163
    
0-100: 0.0189